In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from IPython.display import clear_output
import seaborn as sns

import cPickle as pk
import pandas as pd
import networkx as nx
import scipy.sparse as sp
import itertools as itls
from itertools import combinations
import numpy as np
from math import sqrt

import sys

## Importing
[to top](#Table-of-Contents) 

In [2]:
pwd

u'/Users/innovation-sfi/Dropbox/Research/Innovation/EPO/Data_processing/Python'

In [3]:
df = pk.load(open('pickles_and_munged_csv/pre1950_cated_cumed_codes_as_ints_v1.pk', 'rb')); df.head()

appln_auth,AT,AU,BE,CA,CH,DE,DK,ES,FI,FR,...,IE,IS,IT,JP,LU,NL,NO,SE,SU,US
pub_year,,,,,,,,,,,,,,,,,,,,,
1836,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],"[[1513, 1789], [1268, 1855], [1762, 1821], [24..."
1837,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],"[[5834, 5836], [4907, 5440], [799, 5648], [602..."
1838,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],"[[1537, 1774], [1154, 1208], [2724, 2748], [40..."
1839,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],"[[4212, 4231], [5078, 6180], [4915, 5001], [33..."
1840,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],"[[2722, 2748], [1862, 1862], [1161, 5985], [48..."


In [4]:
s_US = df.US

In [5]:
del df

In [6]:
s_US

pub_year
1836    [[1513, 1789], [1268, 1855], [1762, 1821], [24...
1837    [[5834, 5836], [4907, 5440], [799, 5648], [602...
1838    [[1537, 1774], [1154, 1208], [2724, 2748], [40...
1839    [[4212, 4231], [5078, 6180], [4915, 5001], [33...
1840    [[2722, 2748], [1862, 1862], [1161, 5985], [48...
1841    [[5986, 6001], [3027, 5586], [2964, 3002], [18...
1842    [[4897, 5004], [5740, 5803], [5349, 5421], [11...
1843    [[1762, 1821], [1762, 1821], [5704, 5807], [57...
1844    [[3764, 3825], [1520, 1764], [4641, 4643], [11...
1845    [[5318, 5404], [1060, 4759], [3969, 4052], [23...
1846    [[4097, 6441], [5610, 6516], [1204, 1758], [17...
1847    [[4097, 6441], [1816, 1817], [1722, 5438], [17...
1848    [[335, 2889], [4315, 4750], [5471, 5479, 5479]...
1849    [[5479, 5479], [1496, 1622], [1237, 5473], [50...
1850    [[1754, 1764], [659, 813], [1513, 1789], [1848...
1851    [[327, 332], [4810, 5731], [1829, 1853], [5740...
1852    [[5687, 6199], [4898, 4929], [110, 1211], [489...
1853 

# Applying

In [7]:
def compute_Jac_US_sims(s_US, years):
    
    # create dataframe to store similarities
    df_temp = pd.DataFrame()
    
    # for countries in dataset
    for i, y1 in enumerate(years):
        print '\n%s' % y1 # to see progress
    
            
        # create adjacency dataframe df1 (via nx.graph) for country1
        g1 = nx.MultiGraph()
        for ls in s_US.loc[y1]:
            g1.add_edges_from(combinations(ls, 2))

        # for countries 'greater' than country1 in dataset 
        for y2 in years[i:]:

            # if countries are the same then set similarity to 1
            if y1 == y2:
                print 'self',
                df_temp.loc[y1,y2] = 1

            # else if the country of not empty then calculate DeltaCon sim for country1 and country2
            else:
                print y2, # to see progress

                # create adjacency dataframe df2 (via nx.graph) for country2
                g2 = nx.MultiGraph()
                for ls in s_US.loc[y2]:
                    g2.add_edges_from(combinations(ls, 2))    
                
                lnk_c1 = Counter(g1.edges())
                lnk_c2 = Counter(g2.edges())
                jac_indx = sum((lnk_c1 & lnk_c2).values())/float(sum((lnk_c1 | lnk_c2).values()))

                #assign to dataframe
                df_temp.loc[y1,y2] = jac_indx
                df_temp.loc[y2,y1] = jac_indx
                    
    return df_temp

In [8]:
# initialise 
years = range(1836,1901)
USdf = compute_Jac_US_sims(s_US, years)


1836
self 1837

NameError: global name 'Counter' is not defined

In [ ]:
pk.dump(USdf, open('pickles_and_munged_csv/sim_dfs/all_grant_no_priors_pre1950_jac_multi_sim_US_1836_1900.pk',
                            'wb'))

In [ ]:
USdf

In [ ]:
sns.heatmap(USdf)